Муртазалиев Матвей, 466797, J3110


## 0. Загрузка очищенных данных

In [1]:
import os, gc, math, random, re, json, warnings
import pandas as pd, numpy as np
from tqdm.notebook import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from rank_bm25 import BM25Okapi
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
import nltk

warnings.filterwarnings('ignore')

# Скачиваем словари при первом запуске
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)

True

In [2]:
# Чтение файлов
DATA_DIR = '../data'
questions = pd.read_csv(os.path.join(DATA_DIR, 'Questions_cleared.csv'), encoding='latin1', parse_dates=['CreationDate', 'ClosedDate'])
answers = pd.read_csv(os.path.join(DATA_DIR, 'Answers_cleared.csv'), encoding='latin1', parse_dates=['CreationDate'])
tags = pd.read_csv(os.path.join(DATA_DIR, 'Tags.csv'), encoding='latin1')

print('Questions:', questions.shape)
print('Answers  :', answers.shape)
print('Tags     :', tags.shape)


Questions: (1264216, 11)
Answers  : (2014516, 7)
Tags     : (3750994, 2)


In [3]:
questions.head()

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body,AnswerCount,TagList,CleanBody,CleanTitle
0,80,26.0,2008-08-01 13:57:07+00:00,NaT,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,3,"['flex', 'actionscript-3', 'air']",write database generation script sql want exec...,sqlstatement execute multiple query one statement
1,90,58.0,2008-08-01 14:41:24+00:00,2012-12-26 03:45:49+00:00,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...,3,"['svn', 'tortoisesvn', 'branch', 'branching-an...",really good tutorial explain branching merge a...,good branching merge tutorial tortoisesvn
2,120,83.0,2008-08-01 15:50:08+00:00,NaT,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,1,"['sql', 'asp.net', 'sitemap']",anyone get experience create sql base asp net ...,asp net site map
3,180,2089740.0,2008-08-01 18:42:19+00:00,NaT,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...,9,"['algorithm', 'language-agnostic', 'colors', '...",something pseudo solve many time never quite f...,function create color wheel
4,260,91.0,2008-08-01 23:22:08+00:00,NaT,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,9,"['c#', '.net', 'scripting', 'compiler-construc...",little game write us database back end trading...,add script functionality net application


In [4]:
answers.head()

,Id,OwnerUserId,CreationDate,ParentId,Score,Body,CleanBody
0,92,61.0,2008-08-01 14:45:37+00:00,90,13,"<p><a href=""http://svnbook.red-bean.com/"">Vers...",version control subversion good resource sourc...
1,124,26.0,2008-08-01 16:09:47+00:00,80,12,<p>I wound up using this. It is a kind of a ha...,wound use kind hack actually work pretty well ...
2,199,50.0,2008-08-01 19:36:46+00:00,180,1,<p>I've read somewhere the human eye can't dis...,read somewhere human eye distinguish less valu...
3,269,91.0,2008-08-01 23:49:57+00:00,260,4,"<p>Yes, I thought about that, but I soon figur...",yes thought soon figure another domain specifi...
4,307,49.0,2008-08-02 01:49:46+00:00,260,28,"<p><a href=""http://www.codeproject.com/Article...",oleg shilo script solution code project really...


## 1. Индексирование

### TF-IDF

In [5]:
# Ограничение на количество документов (вопросов) (None = взять все)
MAX_DOCS = None

# Используем очищенный текст из ЛР3
docs = questions['CleanBody'] if MAX_DOCS is None else questions['CleanBody'][:MAX_DOCS]

# Заменяем nan на пустую строку
docs = docs.fillna("")

# Связь документов и векторов (связь между id вопроса и строкой в матрице)
id2row = docs.index.to_list()  # row -> question_id
row2id = {r: i for i, r in enumerate(id2row)}  # question_id -> row

# Создаем объект TF-IDF векторизатора
# Аргументы: 
# 1 и 2. Мин и Макс количество документов для включения терма
# 3. Диапазон n-грамм из текста. Например тут означает, что будут извлекаться как униграммы (1-сложные слова), так и биграммы (2-сложные слова, то есть пары слов)
tfidf = TfidfVectorizer(min_df=3, max_df=0.9, ngram_range=(1, 2))

# Строим разреженную TF-IDF матрицу для документов (строка соответствует документу (вопросу), колонка - терму)
# Значение ячейки это вес ее терма
# Это и есть основной индекс для быстрого поиска, так как позже будем считать косинусное сходство между вектором запроса и X
# Инвертированный индекс (то есть индексы вопросов в которых встречается конкретный терм) реализованы внутри, как и в BM25
X = tfidf.fit_transform(docs)
print(f'Sparse TF-IDF matrix shape: {X.shape}')


Sparse TF-IDF matrix shape: (1264216, 1932765)


### BM25

In [7]:
# Токенизируем корпус для BM25 (список списков слов)
tokenized_corpus = [d.split() for d in docs]

# Строим BM25 по токенизированным документам
bm25 = BM25Okapi(tokenized_corpus)
# Получим среднее число токенов в документе
avgdl = bm25.avgdl

# Веса термов BM25
# print(bm25.get_scores())

print(f'BM25 avgdl = {avgdl:.1f}')

BM25 avgdl = 45.0


## 2. Реализация поиска

In [8]:
STOPWORDS = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()


# Функция для определения части речи
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {
        'J': wordnet.ADJ,
        'N': wordnet.NOUN,
        'V': wordnet.VERB,
        'R': wordnet.ADV
    }
    return tag_dict.get(tag, wordnet.NOUN)


# Предобработка запроса теми же шагами
def preprocess_query(q: str):
    # Убираем теги
    text = re.sub(r'<code.*?>.*?</code>', '', q, flags=re.DOTALL | re.IGNORECASE)
    text = re.sub(r'<[^>]+>', ' ', text)
    # В нижний шрифт
    text = text.lower()
    # Убираем markdown и ссылки
    text = re.sub(r'http\S+|www\S+', ' ', text)
    # Оставляем только буквы/цифры
    text = re.sub(r'[^a-z0-9\s]', ' ', text)
    # Преобразуем в начальную форму
    tokens = [lemmatizer.lemmatize(tok, get_wordnet_pos(tok)) for tok in text.split() if tok not in STOPWORDS and len(tok) > 2]
    return ' '.join(tokens), tokens


def search_tfidf(query: str, top_k: int = 10):
    # Предобработка
    clean_q, _ = preprocess_query(query)
    # Векторизируем
    q_vec = tfidf.transform([clean_q])
    # Смотрим сходство запроса с каждым вопросом
    sims = cosine_similarity(q_vec, X).ravel()
    # Возвращаем top_k схожих вопросов 
    top_idx = sims.argsort()[::-1][:top_k]
    return [(id2row[i], sims[i]) for i in top_idx if sims[i] > 0]


def search_bm25(query: str, top_k: int = 10):
    # Предобработка
    _, toks = preprocess_query(query)
    # Скоры для каждого токена
    scores = bm25.get_scores(toks)
    # Возвращаем top_k схожих вопросов 
    top_idx = np.argsort(scores)[::-1][:top_k]
    return [(id2row[i], scores[i]) for i in top_idx if scores[i] > 0]

## 3. Извлечение ответа

### Столбец ответов

In [62]:
def get_best_answer(question_id):
    subset = answers[answers['ParentId'] == question_id]

    # Если пусто, то ответов нет
    if subset.empty:
        return None

    # Берем лучший по рейтингу
    best = subset.sort_values(['Score', 'CreationDate'], ascending=[False, True]).iloc[0]
    return best['CleanBody']

In [63]:
# Добавляем столбец AcceptedAnswer
questions['AcceptedAnswer'] = questions['Id'].map(get_best_answer)
questions.head()

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body,AnswerCount,TagList,CleanBody,CleanTitle,AcceptedAnswerId,AcceptedAnswer
0,80,26.0,2008-08-01 13:57:07+00:00,NaT,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,3,"['flex', 'actionscript-3', 'air']",write database generation script sql want exec...,sqlstatement execute multiple query one statement,wound use kind hack actually work pretty well ...,wound use kind hack actually work pretty well ...
1,90,58.0,2008-08-01 14:41:24+00:00,2012-12-26 03:45:49+00:00,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...,3,"['svn', 'tortoisesvn', 'branch', 'branching-an...",really good tutorial explain branching merge a...,good branching merge tutorial tortoisesvn,easy click click instruction specific tortoise...,easy click click instruction specific tortoise...
2,120,83.0,2008-08-01 15:50:08+00:00,NaT,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,1,"['sql', 'asp.net', 'sitemap']",anyone get experience create sql base asp net ...,asp net site map,jeff prosise version msdn magazine work pretty...,jeff prosise version msdn magazine work pretty...
3,180,2089740.0,2008-08-01 18:42:19+00:00,NaT,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...,9,"['algorithm', 'language-agnostic', 'colors', '...",something pseudo solve many time never quite f...,function create color wheel,first thought generate vector space maximize d...,first thought generate vector space maximize d...
4,260,91.0,2008-08-01 23:22:08+00:00,NaT,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,9,"['c#', '.net', 'scripting', 'compiler-construc...",little game write us database back end trading...,add script functionality net application,oleg shilo script solution code project really...,oleg shilo script solution code project really...


In [64]:
# Сохраняем очищенные данные
questions.to_csv(DATA_DIR + '/Questions_cleared.csv', index=False)

### Тестирование

In [61]:
def test_engines(query):
    print(f'Query: {query}\n')

    for engine, func in [('TF-IDF', search_tfidf), ('BM25', search_bm25)]:
        print(f'== {engine} ==')
        for qid, score in func(query, top_k=1):
            title = questions.loc[qid]['Title'][:120]
            print(f'    [{qid}] {score:0.4f}  —  {title}')
            best_ans = questions.loc[qid]['AcceptedAnswer'][:120]
            if best_ans == '':
                print("    No answers found for this question")
                return
            print(f'    Best answer: {best_ans}\n')

In [59]:
test_engines('How to convert a string to a list in Python?')

Query: How to convert a string to a list in Python?

== TF-IDF ==
    [424043] 0.4350  —  Convert string to list in python
    Best answer: without problem seem problem either

== BM25 ==
    [424043] 20.5504  —  Convert string to list in python
    Best answer: without problem seem problem either



In [60]:
test_engines('How to convert string to int in Python?')

Query: How to convert string to int in Python?

== TF-IDF ==
    [967922] 0.4233  —  Is there other ways to convert a string to int in python2 without int and string.atoi?
    Best answer: would recommend use try catch also use module

== BM25 ==
    [700306] 22.6460  —  ValueError: invalid literal for int() with base 10: '0.00'
    Best answer: easy way first convert sure fractional part always zero faster would use float



## 4. Оценка качества

Формулы для оценки качества и поиска рекомендашек. Они измеряют насколько хорошо система возвращает релевантные документы на запросы
1. Точность на топ-k результатах (по количеству релевантных в топе)
2. Mean Reciprocal Rank измеряет, насколько быстро (раньше) среди результатов встречается первый релевантный документ


In [11]:
from IPython.display import Markdown

Markdown(r'''
$$
\text{Precision@}k = \frac{1}{k} \sum_{i=1}^{k} \mathbf{1}[\text{rel}_i]
$$
- где $\mathbf{1}[\text{rel}_i]$ — индикатор релевантности результата на $i$-ой позиции

$$
\text{MRR} = \frac{1}{|Q|} \sum_{q \in Q} \frac{1}{\text{rank}_q}
$$
- где $\text{rank}_q$ — позиция первого релевантного документа для запроса $q$
''')


$$
\text{Precision@}k = \frac{1}{k} \sum_{i=1}^{k} \mathbf{1}[\text{rel}_i]
$$
- где $\mathbf{1}[\text{rel}_i]$ — индикатор релевантности результата на $i$-ой позиции

$$
\text{MRR} = \frac{1}{|Q|} \sum_{q \in Q} \frac{1}{\text{rank}_q}
$$
- где $\text{rank}_q$ — позиция первого релевантного документа для запроса $q$


In [44]:
# Подготовка случайного набора запросов
SAMPLE_N = 100
sample_ids = random.sample(list(questions.index), SAMPLE_N)
sample_queries = questions.loc[sample_ids]['CleanTitle'].tolist()


# Функция оценки поиска
def evaluate(engine_func):
    k = 5  # Сколько документов из топа будем брать
    precisions, recip_ranks = [], []

    for qid, q in zip(sample_ids, sample_queries):
        # Выполняем поиск по запросу
        results = [doc for doc, _ in engine_func(q, top_k=k)]

        # Если не нашли, то нули
        if not results:
            precisions.append(0)
            recip_ranks.append(0)
            continue

        # Строим список релевантности
        rel = [1 if doc == qid else 0 for doc in results]

        # Считаем точность
        precisions.append(sum(rel) / k)

        # Считаем ранк для MRR
        try:
            rank = rel.index(1) + 1  # Позиция первого релевантного ответа
            recip_ranks.append(1 / rank)
        except ValueError:
            recip_ranks.append(0)  # Нет релевантных документов

    # Возвращаем среднее по всем запросам
    return np.mean(precisions), np.mean(recip_ranks)


# Вычисляем метрики
p_tfidf, mrr_tfidf = evaluate(search_tfidf)
p_bm25, mrr_bm25 = evaluate(search_bm25)

# Результаты
print(f'TF-IDF  |  Precision@5 = {p_tfidf:.2f},   MRR = {mrr_tfidf:.2f}')
print(f'BM25    |  Precision@5 = {p_bm25:.2f},   MRR = {mrr_bm25:.2f}')

TF-IDF  |  Precision@5 = 0.07,   MRR = 0.29
BM25    |  Precision@5 = 0.10,   MRR = 0.41


## 5. Документация результатов

* **TF-IDF** обеспечивает простой и быстрый поиск без сложных вычислительных ресурсов.

  Формула TF-IDF для термина $t$ в документе $d$:
  $$
  \text{TF-IDF}_{t,d} = \text{TF}_{t,d} \cdot \log \left( \frac{N}{1 + n_t} \right)
  $$
  где:
  - $\text{TF}_{t,d}$ — частота термина $t$ в документе $d$,
  - $N$ — общее число документов,
  - $n_t$ — количество документов, в которых встречается термин $t$.

  TF-IDF увеличивает вес редких, но потенциально значимых слов, и уменьшает влияние часто встречающихся слов (например, "data", "the").

* **BM25** обычно даёт более релевантные результаты благодаря учёту длины документа и сглаживанию TF.

  Формула оценки BM25:
  $$
  \text{score}(q, D) = \sum_{t \in q} \text{IDF}_t \cdot \frac{f(t,D) \cdot (k_1 + 1)}{f(t,D) + k_1 \cdot (1 - b + b \cdot \frac{|D|}{\text{avgdl}})}
  $$
  где:
  - $f(t,D)$ — частота термина $t$ в документе $D$,
  - $|D|$ — длина документа,
  - $\text{avgdl}$ — средняя длина документов в коллекции,
  - $k_1$, $b$ — гиперпараметры (обычно $k_1 = 1.5$, $b = 0.75$),
  - $\text{IDF}_t = \log \left( \frac{N - n_t + 0.5}{n_t + 0.5} + 1 \right)$

  В отличие от TF-IDF, BM25 ограничивает вклад термина с очень высокой частотой и корректирует результат в зависимости от длины документа.

* **Ограничения классических методов**:
  * отсутствие учёта **синонимии и семантики**: например, "car" и "automobile" не распознаются как близкие;
  * невозможность **понимать порядок слов** (большинство моделей представляют тексты как мешок слов);
  * **чувствительность к формулировке запроса** — небольшое изменение формулировки может сильно изменить результаты.
